In [1]:
%matplotlib inline
from IPython.display import Image

# 強化學習Cartpole (tensorflow+keras)

相較於之前學習過的機器視覺與自然語言，強化學習是另一種不同的人工智能方向。機器視覺與語言注重的是如何透過神經網路結構來實現人類視覺以及語言理解的內部機制。但是強化學習卻不關心這些，強化學習的本質是一種行為控制的科學，它關心的是如何能夠控制智能體的行為模式，但卻不需要關注於智能體實現這個行為模式的內部結構。舉例來說，傳統的技術如果要讓智能體懂得「跑」這件事，他需要對於人類的運動機制、肌肉控制等過程需要有巨細靡遺理解，但是也正是這些阻礙地讓機器跑起來的可能性，因為這裡涉及到太多的內部機制。而強化學習的目標則是透過外部的獎賞與懲罰，透過智能體與環境間的交互，來誘導智能體自然產生「跑」這個行為。


![md_images](../Images/rl.jpg)

## CartPole: 數據與環境

在本次實作中，我們將會來介紹強化學習的經典案例Cartpole。我們將使用OpenAI's [gym](https://github.com/openai/gym)模擬器中的[CartPole](https://gym.openai.com/envs/CartPole-v0)環境來做為我們智能體所存在以及交互的世界，它會不斷的反饋給我們木棒與台車的狀態。

環境給予我們的狀態(state)是一個長度為4的向量 $(x, \dot{x}, \theta, \dot{\theta})$, 分別表示 *台車位置*, *台車速度*, *木棒對垂直交角*, *木棒角速度(落下速度)*,

### CartPole: 行動策略
至於台車為了不讓棒子掉落，必須採取對應行動，它能夠執行的行動只有兩種: `向左` 或 `向右`

### CartPole: 環境交互
當台車執行了某個行動後，它可從環境中獲得
  * 每多存活一刻就會獲得獎賞 +1 
  * 新狀態 $(x', \dot{x}', \theta', \dot{\theta}')$

### CartPole: 失敗的定義
這也正是「活得越久，領得越多」，那麼機器的目標很明確，就是希望採取正確的行動，來讓木棒可以撐久一點不要掉下去。那至於甚麼樣的情況會導致木棒掉落呢？如果觸發以下條件，則episode終止
 * 木棒距垂直角度大於15度
 * 臺車移動距離中心超過2.4個單位

### CartPole: 成功的定義
我們對於台車的任務給予一個任務完成認定條件如下：
 * 智能體在過去50個episodes期間累積獎賞值200以上

### CartPole: 優化的目標
如果用RL的術語來說，整個模型的目標就是要找尋 _行動策略(向左向右)_ $a$, 藉由與環境的互動(讓木棒保持平衡)好讓 _獎賞_ $r$ 最大化。於是給定一系列的實驗 $$s \xrightarrow{a} r, s'$$ 讓後讓智能體學會在給定的狀態下 $s$ 找出最佳的行動策略 $a$ 以將跨時間累積獎賞最大化 $r$ :

$$
Q(s,a) = r_0 + \gamma r_1 + \gamma^2 r_2 + \ldots = r_0 + \gamma \max_a Q^*(s',a)
$$

此處的 $\gamma \in [0,1)$ 是波爾曼方程式 [*Bellmann*-equation] 用來控制評估未來獎賞的折價因子，我們也稱之為(https://en.wikipedia.org/wiki/Bellman_equation).

在接下來的範例中，我們將示範如何針對狀態空間建模，以及如何根據收到的獎賞，來轉化為取得未來最大獎賞的行動。

![md_images](../Images/polecart.gif)

在此我們將採用兩種常見的技術:

**Deep Q-Networks (DQN)**: DQNs在2015年只透過遊戲畫面像素數據就能訓練智能體玩Atari電動遊戲而聲名大噪。我們訓練神經網路學習 $Q(s,a)$ 值 (也就是 $Q$-Network )，你可以把Q函數想像為一個可以根據目前狀態去查詢各個行動策略對應的可能獎賞值(目前獎賞與預估未來獎賞)的對應表，因為這個表可能很複雜或是根本不存在，所以我們需要用一個網路去近似它。根據這 $Q$ 函數值，我們交會逐一評估所有可能的策略，選擇獎賞最高的為最佳策略。(我常開玩笑說DQN是最視錢如命的模型，它的輸入是視覺，輸出是怎麼做會最賺錢)，它的輸出既然是獎賞的期望值，所以它的本質是一種迴歸。

**Policy gradient (策略梯度)**: 這個方法是在神經網路中直接估計策略(行動組合)的機率分布，通過機率選擇動作的子集來最大化獎勵，所以你可以把它視為是一種分類模型，輸出的是各個行動的分布機率。 

請注意，由於這個實作的目的是為了要理解RL的概念，因此我們網路部分都使用簡單的淺層網路，當然這個部分是可以日後再擴充與使用結構更複雜的網路。

In [2]:
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
import seaborn as sns
import math 
import os 
import random
import copy
from collections import deque
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import Adam


style.use('ggplot')
%matplotlib inline


Using TensorFlow backend.


執行實作前需要安裝 OpenAI gym包

In [3]:
try:
    import gym
except:
    !pip install gym
    import gym

## 強化學習方法1: DQN

DQN的主要核心在於收集歷史案例的四元組$(s,a,r,s')$ ，這四個分別是之前的狀態，執行的行動、得到的獎賞以及執行後的新狀態，透過這四元祖，我們可以利用深度學習網路去近似價值函數$Q(s,a)$，這個價值函數$Q(s,a)$的實際值應該要接近 $r+\gamma \max_{a'}Q(s',a')$, 其中 $\gamma$ 是未來獎勵的折扣因子，值介於0和1之間。這種先收集樣本，然後在採樣後建模訓練的手法稱之為「回放(replay)」，可以有效地換解如果我們按照數據的時序訓練時，容易偏重當下的案例，反而造成樣本的不均衡。


### Model: DQN

$$
l_1 = relu( x W_1 + b_1) \\
Q(s,a) = l_1 W_2 + b_2 \\
$$

來源參考自Keras版本實現, https://github.com/jaara/AI-blog/blob/master/CartPole-basic.py, 作者是Jaromír Janisch 首見於他的 [AI blog](https://jaromiru.com/2016/09/27/lets-make-a-dqn-theory/)。

首先利用gym建置CartPole-v0模擬器環境，狀態空間長度為4，行動策略長度為2

STATE_COUNT = 4 (對應至 $(x, \dot{x}, \theta, \dot{\theta})$),
ACTION_COUNT = 2 (對應至`向左` or `向右`)

In [4]:
env = gym.make('CartPole-v0')
STATE_COUNT = env.observation_space.shape[0]
ACTION_COUNT = env.action_space.n

STATE_COUNT, ACTION_COUNT

(4, 2)

在此，我們使用簡單的兩層全連接層網路來做示範，用來逼近價值函數$Q(s,a)$。如果你覺得兩層太簡單了，你也可以自行設計或者是置換為更複雜的網路結構。

![md_images](../Images/dqn_network.jpg)

In [5]:
# 目標獎賞
REWARD_TARGET =  200
# 平均episodes數
BATCH_SIZE_BASELINE = 50

class DQNAgent:
    def __init__(self, STATE_COUNT, ACTION_COUNT):
        self.state_size = STATE_COUNT
        self.action_size = ACTION_COUNT
        #記憶(Memory) 類別是專門用來儲存歷史的四元組紀錄。(之前的狀態，執行的行動、得到的獎賞以及執行後的新狀態)
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # 折價率
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # 利用三層的全聯階層近似Q函數
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64,  activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    #記憶(Memory) 類別是用來儲存不同的狀態、行動以及獎賞
    #儲存 ( s, a, r, s_ )
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(np.reshape(state,(1,len(state))))
        return np.argmax(act_values) 
    #智能體 重放(replay)過去的行動以訓練出能讓獎賞最大化的行動集合。
    def replay(self, batch_size):
        #從記憶中隨機取n筆
        minibatch = random.sample(self.memory, batch_size)
        x = np.zeros((batch_size, STATE_COUNT)).astype(np.float32)
        y = np.zeros((batch_size, ACTION_COUNT)).astype(np.float32)
        
        #組出批次的x,y
        for i in range(batch_size):
            state, a, r, next_state, done = minibatch[i]
            p = self.model.predict(np.reshape(state,(1,len(state))))
            p_ = self.model.predict(np.reshape(next_state,(1,len(next_state))))
       
            t = p[0]
            if not done:
                t[a] = r + GAMMA * np.amax(p_[0])
            else:
                t[a] = r

            x[i] = state
            y[i] = t
            
        self.model.fit(x, y, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

 `記憶(Memory)` 類別是專門用來儲存歷史的四元組紀錄。(之前的狀態，執行的行動、得到的獎賞以及執行後的新狀態)

`智能體` 使用 `Brain` 以及 `Memory` 來重放(replay)過去的行動以選擇能讓獎賞最大化的行動集合。

In [ ]:
MEMORY_CAPACITY = 100000
BATCH_SIZE = 64

GAMMA = 0.9 # 折價因子

MAX_EPSILON = 1  #初期還沒有案例可以供建模，因此100%根據隨機案例
MIN_EPSILON = 0.01 # 即使模型準確率越來越高，還是必須保留部分比例基於隨機案例
LAMBDA = 0.0001    # 衰減速度

TOTAL_EPISODES = 3000

agent = DQNAgent(STATE_COUNT, ACTION_COUNT)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

def run(agent):
    #將環境重置
    state = env.reset()
    #獎賞歸零
    R = 0

    while True:
        #env.render()指令會幫我們渲染出目前台車狀態圖形
        env.render()
        
        #agent.act(s.astype(np.float32))表示是根據目前狀態產生對應行動
        #在這邊act函數初期是基於隨機行動，後面隨機行動站筆會逐漸下降
        action = agent.act(state)
        #根據此行動，產出對應的獎賞以及新的狀態
        next_state, reward, done, _ = env.step(action)
        if done: # 若最後是終止狀態(倒下)，則新狀態為None
            next_state= np.zeros(STATE_COUNT)
        #構成四元組儲存在記憶中
        agent.remember(state, action, reward, next_state, done)
        #根據隨機抽樣進行回放訓練
        agent.replay(min(batch_size, len(agent.memory)))
        #將狀態更新至新狀態
        state = next_state
        #將獎賞累加
        R += reward
        # 若最後是終止狀態(倒下)，則獎賞不再更新
        if done:
            return R
        

In [ ]:
reward_sum = 0
for e in range(TOTAL_EPISODES):
    reward_sum+=run(agent)
    if (e+1) % 50 == 0:
        print("episode: {}/{}, 平均獎賞為: {}".format(e+1, TOTAL_EPISODES, reward_sum/50))
        reward_sum=0
        agent.save("Models/dqn_keras.h5")
        if reward_sum/50>250:
            break
  

episode: 50/3000, 平均獎賞為: 18.82
episode: 100/3000, 平均獎賞為: 157.72
episode: 150/3000, 平均獎賞為: 170.2
episode: 200/3000, 平均獎賞為: 172.96
episode: 250/3000, 平均獎賞為: 167.22
episode: 300/3000, 平均獎賞為: 163.28
